<a href="https://colab.research.google.com/github/kiranmuloor/DataScience/blob/master/ClassWork/Keras_LSTM_text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#import required libraries
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils


In [9]:
#Load the file
# load text and covert to lowercase
filename = "wonderland.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()
raw_text[0:100]

"\ufeffproject gutenberg's alice's adventures in wonderland, by lewis carroll\n\nthis ebook is for the use o"

In [0]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [11]:
chars[0:10]

['\n', ' ', '!', '"', '#', '$', '%', "'", '(', ')']

In [12]:
char_to_int

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 '#': 4,
 '$': 5,
 '%': 6,
 "'": 7,
 '(': 8,
 ')': 9,
 '*': 10,
 ',': 11,
 '-': 12,
 '.': 13,
 '/': 14,
 '0': 15,
 '1': 16,
 '2': 17,
 '3': 18,
 '4': 19,
 '5': 20,
 '6': 21,
 '7': 22,
 '8': 23,
 '9': 24,
 ':': 25,
 ';': 26,
 '?': 27,
 '@': 28,
 '[': 29,
 ']': 30,
 '_': 31,
 'a': 32,
 'b': 33,
 'c': 34,
 'd': 35,
 'e': 36,
 'f': 37,
 'g': 38,
 'h': 39,
 'i': 40,
 'j': 41,
 'k': 42,
 'l': 43,
 'm': 44,
 'n': 45,
 'o': 46,
 'p': 47,
 'q': 48,
 'r': 49,
 's': 50,
 't': 51,
 'u': 52,
 'v': 53,
 'w': 54,
 'x': 55,
 'y': 56,
 'z': 57,
 '\ufeff': 58}

In [13]:
#Summarize the Dataset
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  163781
Total Vocab:  59


In [14]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  163681


In [0]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [16]:
X

array([[[0.98305085],
        [0.79661017],
        [0.83050847],
        ...,
        [0.61016949],
        [0.01694915],
        [0.77966102]],

       [[0.79661017],
        [0.83050847],
        [0.77966102],
        ...,
        [0.01694915],
        [0.77966102],
        [0.62711864]],

       [[0.83050847],
        [0.77966102],
        [0.69491525],
        ...,
        [0.77966102],
        [0.62711864],
        [0.01694915]],

       ...,

       [[0.86440678],
        [0.77966102],
        [0.01694915],
        ...,
        [0.77966102],
        [0.77966102],
        [0.71186441]],

       [[0.77966102],
        [0.01694915],
        [0.66101695],
        ...,
        [0.77966102],
        [0.71186441],
        [0.84745763]],

       [[0.01694915],
        [0.66101695],
        [0.61016949],
        ...,
        [0.71186441],
        [0.84745763],
        [0.22033898]]])

In [17]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [0]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [19]:
X.shape[1]
X.shape[2]

1

In [0]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [23]:
#Fitting the model
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
163681/163681 [==============================] - 309s 2ms/step - loss: 2.0657

Epoch 00001: loss improved from 2.08759 to 2.06574, saving model to weights-improvement-01-2.0657.hdf5
Epoch 2/20
163681/163681 [==============================] - 309s 2ms/step - loss: 2.0448

Epoch 00002: loss improved from 2.06574 to 2.04482, saving model to weights-improvement-02-2.0448.hdf5
Epoch 3/20
163681/163681 [==============================] - 308s 2ms/step - loss: 2.0243

Epoch 00003: loss improved from 2.04482 to 2.02430, saving model to weights-improvement-03-2.0243.hdf5
Epoch 4/20
163681/163681 [==============================] - 307s 2ms/step - loss: 2.0045

Epoch 00004: loss improved from 2.02430 to 2.00453, saving model to weights-improvement-04-2.0045.hdf5
Epoch 5/20
163681/163681 [==============================] - 305s 2ms/step - loss: 1.9873

Epoch 00005: loss improved from 2.00453 to 1.98728, saving model to weights-improvement-05-1.9873.hdf5
Epoch 6/20
163681/163681 [=========

In [0]:
#Generating Text with the trained model
# load the network weights
filename = "weights-improvement-20-2.0876.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [0]:
#reverse mapping from id to chars

int_to_char = dict((i, c) for i, c in enumerate(chars))

In [27]:
import sys
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")

Seed:
" isk or other medium, a
computer virus, or computer codes that damage or cannot be read by
your equip "
r po the pooject gutenberg-tm electronic works in aoy coot thth the pooeect gutenberg-tm electronic works in aoy coot thth the pooeect gutenberg-tm electronic works in aoy coot thth the pooeect gutenberg-tm electronic works in aoy coot thth the pooeect gutenberg-tm electronic works in aoy coot thth the pooeect gutenberg-tm electronic works in aoy coot thth the pooeect gutenberg-tm electronic works in aoy coot thth the pooeect gutenberg-tm electronic works in aoy coot thth the pooeect gutenberg-tm electronic works in aoy coot thth the pooeect gutenberg-tm electronic works in aoy coot thth the pooeect gutenberg-tm electronic works in aoy coot thth the pooeect gutenberg-tm electronic works in aoy coot thth the pooeect gutenberg-tm electronic works in aoy coot thth the pooeect gutenberg-tm electronic works in aoy coot thth the pooeect gutenberg-tm electronic works in aoy coot thth